In [1]:
import numpy as np

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over='ignore') #Ignore warnings about overflow data points

Loading BokehJS ...

In [2]:
# Set up constant
kBT = 0.001 # Boltzmann constant measured interms of chemical potential
E_min = 0
E_max = 10
factor = 0.2
N = 200

# Set up the variable space for 1D plot
En_mu = np.linspace(E_min, E_max, N) #Energy measured in units of chemical potential
f = 1.0/(np.exp( (factor*En_mu-1)/kBT )+1) # Occupation number according to F-D statistics
df = np.gradient(f) # Take the first derivative of the F-D distribution.

In [3]:
# Set up variable space for 2D plot
ex = ey = np.linspace(-E_max,E_max,2*N)
Ex,Ey = np.meshgrid(ex, ey)
Er = np.sqrt(Ex**2 + Ey**2)
ff = 1.0/(np.exp( (factor*Er-1)/kBT )+1)

In [4]:
# Set up 1D plot
plot = figure(plot_height=400, plot_width=400, title="Fermi-Dirac statistics in 1D",
              tools="pan,reset,save,wheel_zoom",
              x_range=[0.0, 10.0], y_range=[0.0, 1.1])
plot.xaxis[0].axis_label='Energy'
plot.yaxis[0].axis_label='First derivative'
r = plot.line(En_mu, f,line_width=3, line_alpha=0.6)

In [5]:
# Set up 2D plot
plot2 = figure(plot_height=400, plot_width=400, title="Fermi-Dirac statistics in 2D",
              tools="pan,reset,save,wheel_zoom",
              x_range=[-10.0, 10.0], y_range=[-10.0, 10.0], tooltips=[("Ex", "$x"), ("Ey", "$y"), ("Probability", "@image")])
plot2.xaxis[0].axis_label='Kx'
plot2.yaxis[0].axis_label='Ky'
rr = plot2.image(image = [ff], x=-10, y=-10, dw=20, dh=20, palette="Spectral11")

#Optional black circle to mark the zero-temperature fermi level
#rrr = plot2.circle(0.0, 0.0,alpha = 0, radius=1.0, level = "underlay")

In [6]:
plot3 = figure(plot_height=400, plot_width=400, title="First derivative",
              tools="pan,reset,save,wheel_zoom",
              x_range=[0.0, 10.0], y_range=[0.0, 0.6])
plot3.xaxis[0].axis_label='Energy'
plot3.yaxis[0].axis_label='Occupation number'
rrr = plot3.line(En_mu, -df,line_width=3, line_alpha=0.6)

In [7]:
# Set up callbacks to update the 1D live graph
def update_data(kBT):
    # Generate the new curve
    En_mu = np.linspace(E_min, E_max, N) #Energy measured in units of chemical potential
    r.data_source.data['y'] = 1.0/(np.exp( (factor*En_mu-1)/kBT )+1)

    Ex,Ey = np.meshgrid(np.linspace(-E_max,E_max,2*N), np.linspace(-E_max,E_max,2*N))
    Er = np.sqrt(Ex**2 + Ey**2)
    rr.data_source.data['image'] = [1.0/(np.exp( (factor*Er-1)/kBT )+1)]
    
    rrr.data_source.data['y'] = -np.gradient(r.data_source.data['y'])
    push_notebook()

In [8]:
show(row(plot,plot3),notebook_handle=True)
# show(column([row(plot, plot3), plot2]), notebook_handle=True)

In [9]:
interact(update_data, kBT=(0.001,0.2,0.002))

interactive(children=(FloatSlider(value=0.099, description='kBT', max=0.2, min=0.001, step=0.002), Output()), …

<function __main__.update_data(kBT)>